# Domain Relevance Evaluation

Comparing different methods to get domain relevant terms 

In [1]:
from tqdm import tqdm
from parts import collect, oie, domain_relevance, preprocessing

## Initial Load of Background Domains

In [ ]:
# adac domain

def get_corpus(topic, domain, limit, export):
    corpus = []
    path = "resources/txt/"
    counter = 0
    links = collect.get_links(topic, domain, limit)
    for link in tqdm(links):
        if collect.get_text(link, domain):
            text = " ; ".join(collect.get_text(link, domain))
            if len(text) < 100000 and text:
                corpus.append(text)
                if export:
                    with open(path+domain+"/"+str(counter)+".txt", "w") as file:
                        file.write(text)
                    counter += 1
    return corpus

In [ ]:
adac_corpus = get_corpus(0,"adac",0,1)

In [ ]:
chefkoch_corpus = get_corpus("https://www.chefkoch.de/forum/1,27/Haus-Garten.html","chefkoch",5,1)

In [ ]:
car_corpus = get_corpus("https://www.motor-talk.de/forum/audi-80-90-100-200-v8-b158.html","car",3,1)

## Load Background Domains and Extract Terms

In [85]:
from collections import Counter

def clean_uniques(terms):
    clean_terms = []
    counter = 0
    tf = Counter(terms)
    for term in tf:
        if tf[term] > 1:
            clean_terms.append(term)
        else:
            counter += 1
    print(counter,"terms removed due to uniques")
    return clean_terms

In [86]:
### import car_domain from files

def load_domain_terms(domain, limit, clean = 0, del_unique = 0):
    counter = 0
    path = "resources/txt/"
    corpus = []
    terms = []
    while counter < limit:
        try:
            with open(path+domain+"/"+str(counter)+".txt", "r") as file:
                corpus.append(file.read())
            counter += 1
        except:
            break
    
    for doc in tqdm(corpus):
        if del_unique:
            doc_terms = clean_uniques(oie.get_terms(doc))
        else:
            doc_terms = oie.get_terms(doc)
        terms.append(doc_terms)
    if clean:    
        terms = preprocessing.post_term_cleaning(terms)
        
    return terms

In [62]:
adac_domain = load_domain_terms("adac", 10000, 1, 0)

100%|██████████| 2524/2524 [01:20<00:00, 31.19it/s]


deleted time references: 0
deleted date references: 287
deleted links: 77
deleted quotes: 63
deleted ireg expressions: 38
deleted abbreviations: 206


In [87]:
car_domain = load_domain_terms("car", 10000, 1, 1)

  2%|▏         | 3/174 [00:00<00:08, 21.04it/s]

33 terms removed due to uniques
8 terms removed due to uniques
72 terms removed due to uniques
9 terms removed due to uniques
15 terms removed due to uniques


  5%|▍         | 8/174 [00:00<00:08, 19.47it/s]

98 terms removed due to uniques
50 terms removed due to uniques
67 terms removed due to uniques


  6%|▌         | 10/174 [00:00<00:13, 12.31it/s]

...

 99%|█████████▉| 172/174 [00:16<00:00,  5.88it/s]

343 terms removed due to uniques


100%|██████████| 174/174 [00:17<00:00,  9.93it/s]

210 terms removed due to uniques
166 terms removed due to uniques


deleted time references: 2
deleted date references: 4
deleted links: 12
deleted quotes: 70
deleted ireg expressions: 13
deleted abbreviations: 51


In [88]:
chefkoch_domain = load_domain_terms("chefkoch", 10000, 1, 1)

  1%|          | 2/279 [00:00<00:24, 11.48it/s]

55 terms removed due to uniques
158 terms removed due to uniques


  1%|          | 3/279 [00:01<01:58,  2.34it/s]

0 terms removed due to uniques


  2%|▏         | 6/279 [00:01<01:28,  3.08it/s]

...

 99%|█████████▉| 276/279 [01:19<00:00,  3.60it/s]

0 terms removed due to uniques


100%|██████████| 279/279 [01:20<00:00,  3.48it/s]

0 terms removed due to uniques
63 terms removed due to uniques
35 terms removed due to uniques


deleted time references: 0
deleted date references: 168
deleted links: 38
deleted quotes: 165
deleted ireg expressions: 109
deleted abbreviations: 94


## Calculate Metrics 

In [89]:
def get_relevancy(terms, metric):
    tf = domain_relevance.get_tf(terms)
    idf = domain_relevance.get_idf(terms)
    tdf = domain_relevance.get_tdf(terms)
    tf_tdf = {}
    
    if metric == "tf":
        return tf
    elif metric == "tf_tdf":
        for term in set([item for sublist in terms for item in sublist]):
            tf_tdf[term] = tf[term]*tdf[term]
        return tf_tdf
    elif metric == "tf_idf":
        for term in set([item for sublist in terms for item in sublist]):
            tf_idf[term] = tf[term]*idf[term]
        return tf_idf

    else:
        return False
    

In [90]:
metric = "tf_tdf"

In [91]:
adac_relevance = get_relevancy(adac_domain, metric)

In [92]:
car_relevance = get_relevancy(car_domain, metric)

In [93]:
chefkoch_relevance = get_relevancy(chefkoch_domain, metric)

## Select Concepts from Background Knowledge

In [123]:
def get_concepts(candidates, car_terms, adac_terms, chefkoch_terms):
    concepts = set()
    counter1 = 0
    counter2 = 0
    
    for candidate in candidates:
        if adac_terms[candidate] and not chefkoch_terms[candidate]:
            concepts.add(candidate)
            counter1 += 1
        if car_terms[candidate] and not chefkoch_terms[candidate]:
            concepts.add(candidate)
            counter2 += 1
        #else:
            #print(candidate)
            
    print(counter1, counter2)
    return concepts

In [124]:
candidates = set([item for sublist in car_domain for item in sublist])

In [125]:
concepts = get_concepts(candidates, car_relevance, adac_relevance, chefkoch_relevance)

322 2005


In [126]:
print(len(candidates), "VS.", len(concepts))

2737 VS. 2005


In [129]:
concepts

{'3000 €',
 'kraftstoffdruckregler',
 'durchführungen',
 'wasserkastenabdeckung',
 'aob',
 'scheuerstelle',
 'zulassungsstelle',
 '19 zöller',
 'bosch classic',
 'jp',
 'zahnstange',
 'audi_90_quattro_20v',
 'inus',
 'dichtringe',
 'classic',
 'weg ziel',
 'kühlmittelanschlüsse',
 'quarantäne',
 'kreislauf',
 '0,5 mm^2',
 'sportlenkrad',
 'moly',
 'regerge',
 'kompression',
 'anfängerauto',
 'ausführlichen anleitung',
 'flitzer',
 'audi90',
 'abschliessen',
 'simple vorderachse',
 'letzten 6 wochen',
 'stellelement',
 'ot markierung',
 'leuchtmittel',
 'mitm',
 '13d lenkrad',
 'ng',
 'kunststoffspachtelmasse',
 'schaumstoffblock',
 'kombieinstrument',
 'warmem motor',
 'ablaufschlauch',
 'pumpenrelais',
 'klimatronik',
 '4 komplette heber',
 'übrigen leitungen',
 'schaufelrad',
 'neueren modellen',
 '2 austauschlenkgetriebe',
 'simmerring',
 'ensetzten',
 'knarren',
 'o-ringe',
 '5er',
 'windschutzscheibenblende',
 'ölfilter adapter',
 'wärmetauscher',
 'zv-pumpe',
 'dranhängen',
 'ot'

In [128]:
#concepts = list(set([item for sublist in adac_domain for item in sublist]))
with open("concepts.txt", "w") as fp:
    fp.writelines('\n'.join(concepts))

In [115]:
with open("concepts.txt", "r") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

content

['3000 €',
 'kraftstoffdruckregler',
 'durchführungen',
 'wasserkastenabdeckung',
 'aob',
 'scheuerstelle',
 'zulassungsstelle',
 '19 zöller',
 'bosch classic',
 'jp',
 'zahnstange',
 'audi_90_quattro_20v',
 'inus',
 'dichtringe',
 'drinne',
 'classic',
 'weg ziel',
 'kühlmittelanschlüsse',
 'quarantäne',
 'kreislauf',
 'italien',
 '0,5 mm^2',
 'sportlenkrad',
 'moly',
 'regerge',
 'laufzeit',
 'kompression',
 'anfängerauto',
 'ausführlichen anleitung',
 'flitzer',
 'modelle',
 'audi90',
 'abschliessen',
 'gruss',
 'simple vorderachse',
 'letzten 6 wochen',
 'stellelement',
 'quelle',
 'ot markierung',
 'leuchtmittel',
 'fehler',
 'mitm',
 '13d lenkrad',
 'ng',
 'kunststoffspachtelmasse',
 'schaumstoffblock',
 'mühen',
 'kombieinstrument',
 'warmem motor',
 'ablaufschlauch',
 'pumpenrelais',
 'klimatronik',
 '4 komplette heber',
 'übrigen leitungen',
 'schaufelrad',
 'neueren modellen',
 '2 austauschlenkgetriebe',
 'simmerring',
 'manschette',
 'ensetzten',
 'knarren',
 'o-ringe',
 '5e

In [136]:
from random import sample

testset = sample(candidates, 500)

In [139]:
with open("testset.txt", "w") as fp:
    fp.writelines('\n'.join(testset))